In [21]:
from gym import Env, spaces
from gym.spaces import Discrete, Box, Dict
import pandas as pd
import numpy as np
import random
import math
from sklearn import preprocessing
from sqlalchemy import create_engine, MetaData
from sqlalchemy import Table, Column, Date, Integer, String, ForeignKey

In [22]:
DATABASE = {
'host': 'localhost',
'port': '8989',
'database': 'AP',
'user': 'root',
'password': 'root'
}

# 连接数据库
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}"\
                    .format(host=DATABASE['host'],port=DATABASE['port'], db=DATABASE['database'], user=DATABASE['user'], pw=DATABASE['password'])\
                    , echo=False)

#连接数据表
agent = 'bruno'
account = 'Guest'
query = "SELECT * FROM `Agent_data` WHERE `Account` = '" + account +"' AND Agent = '" +agent+"'" 
result = engine.execute(query)
info = pd.DataFrame(result.fetchall(),columns=list(result.keys()))

In [23]:

#取會用到的參數名稱

select = ''
para_list = list(result.keys())[12:]
for para in para_list:
    if(info[para][0] == 1):
        select = select+',`'+para+'`'
select = select[1:]

#取數據

DATABASE = {
'host': 'localhost',
'port': '8989',
'database': 'stock_data',
'user': 'root',
'password': 'root'
}

# 连接数据库
engine = create_engine("mysql+pymysql://{user}:{pw}@{host}:{port}/{db}"\
                    .format(host=DATABASE['host'],port=DATABASE['port'], db=DATABASE['database'], user=DATABASE['user'], pw=DATABASE['password'])\
                    , echo=False)

#连接数据表
query = "SELECT "+ select + " FROM `"+ info['stock_num'][0] +"` WHERE `年月日` BETWEEN '" + str(info['Start_date'][0]) +"' AND '" +str(info['End_date'][0])+"'" 
result = engine.execute(query)
data_df = pd.DataFrame(result.fetchall(),columns=list(result.keys()))

In [24]:
def avg_fill(data,col_list):
    for col in col_list:
        na_array = data[col][data[col].isna()].index
        for na in na_array:
            forward = data[col][na-1:na-4].mean()
            backward = data[col][na+1:na+4].mean()
            if(math.isnan(forward) and math.isnan(backward)):
                fill = 0
            elif(math.isnan(forward) or math.isnan(backward)):
                if(math.isnan(forward)):
                    fill = backward
                else:
                    fill = forward
            else:
                fill = (forward + backward)/2 

            data[col][na] = fill
    return data

In [25]:
def calculate_ema(prices, days, smoothing=2):
    ema = [sum(prices[:days]) / days]
    for price in prices[days:]:
        ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
    return ema

def MACD(list_input,prices):
    ema_short = calculate_ema(prices, 12)
    ema_long = calculate_ema(prices, 26)
    len(ema_short)
    dif = []
    for i in range(0,len(prices)-len(ema_short)):
        tmp = ema_short[i]
        ema_short.insert( i, tmp)
    
    for j in range(0,len(prices)-len(ema_long)):
        tmp = ema_long[i]
        ema_long.insert( i, tmp)

    for k in range(0,len(prices)):
        dif.append(ema_short[k] - ema_long[k])

    MACD = calculate_ema(dif, 9)
    for j in range(0,len(prices)-len(MACD)):
        tmp = MACD[i]
        MACD.insert( i, tmp)

    MACD_arr = np.array(MACD).reshape(len(MACD),1)
    list_input = np.hstack((list_input,MACD_arr))
    return list_input

In [26]:
if(info['AVG_fill'][0] == 1):
    data_df = avg_fill(data_df,list(result.keys()))
else:
    data_df.fillna(info['Custom_fill'][0], inplace = True)

/Users/oreo/opt/anaconda3/envs/RL_env/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [27]:
data = data_df.to_numpy()
price = data_df['收盤價(元)'].values
data = MACD(data,price)

if(info["Standardise"][0] == 1):
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaler.fit(data)
    data = scaler.transform(data)

if(info["Normalize"][0] == 1):
    from sklearn.preprocessing import Normalizer
    transformer = Normalizer().fit(data)
    data = transformer.transform(data)

if(info["Scaleing"][0] == 1):
    from sklearn.preprocessing import MinMaxScaler
    scaler = MinMaxScaler()
    scaler.fit(data)
    data = scaler.transform(data)
    
data[data == 0] = 1e-100

In [89]:
class ETFenv(Env):
    
    def __init__(self, data,space_dict,price_key,reward_driver=1, punish_driver = 3, length = 100, stock_num = 1000, interest_rate = 0.05):

        X = data
        self.X = data
        
        price_index = 0
        for key in space_dict:
            if(price_key == key):break
            price_index+=1
        
        global rd
        rd = random.randint(0,len(X)-(length+3))

        self._max_episode_steps = length
        self.state = X[rd]
        self.price = self.X[rd+1][price_index]
        self.next_price = self.X[rd+2][price_index]
        self.interest_rate = interest_rate
        #set length
        self.length = self._max_episode_steps
        #set holding
        self.reward_driver = reward_driver
        self.punish_driver = punish_driver
        self.stock = stock_num
        self.hold_times=0
        self.reward = 0
        self.asset = []
        self.lastday = 0
        self.hold = False
        self.buy = False
        self.sell = False
        
        #Action Buy(0),Sell(1),Hold(2)
        self.action_space = Box(low = -1, high= 1, shape=(1,), dtype=np.float)
        #Price array
        self.observation_space = spaces.Dict(space_dict)
        
    def step(self, action):
        global rd
        self.length -=1
        

        amount = abs(math.floor(self.stock  * action))

        if action > 0:
            
            try:
                reward_do = math.log10(float(self.next_price/self.price)) * amount
            except:
                print(self.next_price)
                print(self.price)
            reward_leave = math.log10(1+float(self.interest_rate*30/365)) * (self.stock - amount) * self.price
            reward_function_sum = reward_do + reward_leave
            self.reward  = reward_function_sum * self.reward_driver if reward_function_sum >0  else reward_function_sum * self.punish_driver
            
        
        elif action < 0:
            reward_do = math.log10(float(self.price/self.next_price)) * amount
            reward_leave = math.log10(1+float(self.interest_rate*30/365)) * (self.stock - amount) * self.price # 可能要改成沒賣的那些股票他們的價值變化
            reward_function_sum = reward_do + reward_leave
            self.reward  = reward_function_sum * self.reward_driver if reward_function_sum >0  else reward_function_sum * self.punish_driver
            #self.reward  = reward_do * self.reward_driver if reward_do >0  else reward_do * self.punish_driver

            #leave_value = (self.stock - amount) * self.next_price
            #next_value = amount * self.price + leave_value
            #this_value = self.price * self.stock
            #reward_function = math.log10(float(this_value/next_value)) 
            #self.reward  = reward_function * self.reward_driver if reward_function >0  else reward_function * self.punish_driver
        
            
        else:
            self.hold = True
        
        if self.length <=0:
            done = True
            info={}
            return self.state, self.reward, done, info
        else:
            done = False
            
        rd +=1
        self.state = self.X[rd]
        self.price = self.X[rd+1][0]
        self.next_price = self.X[rd+2][0]
        info={}
        return self.state, self.reward, done, info
        
    def render(self):
        #Visualization is not work for this project~
        pass
    
    def reset(self): 
        global rd
        self.sell = False
        self.buy = False
        self.hold = False
        self.lastday = 0
        self.asset = []
        self.hold_times= 0
        rd = random.randint(0,len(self.X)-(self._max_episode_steps+3))
        self.state = self.X [rd]
        self.price = self.X[rd+1][price_index]
        self.next_price = self.X[rd+2][price_index]
        self.length = self._max_episode_steps
        return self.state

In [90]:
space_dict = {}
for col in list(result.keys()):
    space_dict[col] = spaces.Box(low=np.array([data_df[col].min()]), high=np.array([data_df[col].max()])) 
    
env = ETFenv(data = data,space_dict=space_dict,price_key = '收盤價(元)',reward_driver= 0.05, punish_driver = 0.05, length = 100, stock_num = 1000, interest_rate = 0.05)

/Users/oreo/opt/anaconda3/envs/RL_env/lib/python3.7/site-packages/gym/logger.py:34: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize("%s: %s" % ("WARN", msg % args), "yellow"))
/Users/oreo/opt/anaconda3/envs/RL_env/lib/python3.7/site-packages/ipykernel_launcher.py:36: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


In [98]:
episodes = 10

for episodes in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action =env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score += reward
    print('Episode:{} Score:{}'.format(episodes, score))

Episode:1 Score:6.606008356321982
Episode:2 Score:-0.7923398784765248
Episode:3 Score:0.7503970977258511
Episode:4 Score:0.9781629544689064
Episode:5 Score:4.602701079725094
Episode:6 Score:1.3420580106281241
Episode:7 Score:2.2322072801897326
Episode:8 Score:1.5937957010757942
Episode:9 Score:-2.2045584192410916
Episode:10 Score:-0.9390494875121607


# TEST AREA

### avg_fill testing

In [62]:
na_array = data['開盤價(元)'][data['開盤價(元)'].isna()].index
for na in na_array:
    forward = data['開盤價(元)'][na-1:na-4].mean()
    backward = data['開盤價(元)'][na+1:na+4].mean()
    if(math.isnan(forward) and math.isnan(backward)):
        fill = 0
    elif(math.isnan(forward) or math.isnan(backward)):
        if(math.isnan(forward)):
            fill = backward
        else:
            fill = forward
    else:
        fill = (forward + backward)/2 
    
    data['開盤價(元)'][na] = fill

/Users/oreo/opt/anaconda3/envs/RL_env/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [99]:
space_dict = {}
i = 0
for col in list(result.keys()):
    space_dict[col] = spaces.Box(low=np.array([data[:,i].min()]), high=np.array([data[:,i].max()]),dtype=np.float64)
    i+=1
    
space_dict['MACD'] = spaces.Box(low=np.array([data[:,i].min()]), high=np.array([data[:,i].max()]),dtype=np.float64)
    
space_dict

{'開盤價(元)': Box([1.e-100], [1.], (1,), float64),
 '最高價(元)': Box([1.e-100], [1.], (1,), float64),
 '最低價(元)': Box([1.e-100], [1.], (1,), float64),
 '收盤價(元)': Box([1.e-100], [1.], (1,), float64),
 '成交量(千股)': Box([1.e-100], [1.], (1,), float64),
 '成交值(千元)': Box([1.e-100], [1.], (1,), float64),
 '報酬率％': Box([1.e-100], [1.], (1,), float64),
 '週轉率％': Box([1.e-100], [1.], (1,), float64),
 '成交筆數(筆)': Box([1.e-100], [1.], (1,), float64),
 '合計買賣超(千股)': Box([1.e-100], [1.], (1,), float64),
 '融資餘額(張)': Box([1.e-100], [1.], (1,), float64),
 '融券餘額(張)': Box([1.e-100], [1.], (1,), float64),
 'MACD': Box([1.e-100], [1.], (1,), float64)}

In [101]:
list(result.keys())

['開盤價(元)',
 '最高價(元)',
 '最低價(元)',
 '收盤價(元)',
 '成交量(千股)',
 '成交值(千元)',
 '報酬率％',
 '週轉率％',
 '成交筆數(筆)',
 '合計買賣超(千股)',
 '融資餘額(張)',
 '融券餘額(張)']